Started comparing the available options to read microscopy file.

- mosaic

In [1]:
%load_ext autoreload
%autoreload 2

#%load_ext line_profiler
%load_ext memory_profiler

lif = "tests/data/2015Aug28_TransHXB2_50min+DMSO.lif"
# C=3 T=4 S=15
img_tile = "tests/data/t4_1.tif"
# C=4 T=3 S=14 scattered
img_void_tile = 'tests/data/tile6_1.tif'
# C=2 T=81
imgsingle = "tests/data/exp2_2.tif"
# C=3 T=7
mcts = "tests/data/multi-channel-time-series.ome.tif"

bt = "tests/data/LC26GFP_1.tf8"

#import matplotlib.pyplot as plt
#import skimage.io
import numpy as np
import tifffile

## ImageIO

In [2]:
from imageio.v3 import imread
%timeit i = imread(img_void_tile, index=13)
i = imread(img_void_tile, index=13)
i.shape

22.9 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


(4, 3, 512, 512)

It can read tif (tf8) files. Series might be passed using `index` (you need to know in advance).

## AICSImageIO

In [3]:
from aicsimageio import AICSImage
%timeit i = AICSImage(img_void_tile)
i = AICSImage(img_void_tile, reconstruct_mosaic=True)
i_lif = AICSImage(lif)

475 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
i_lif.scenes

('Series001', 'Series004', 'Series007', 'Series009', 'Series011')

In [5]:
i.get_dask_stack()

dask.array<stack, shape=(14, 4, 3, 1, 512, 512), dtype=uint16, chunksize=(1, 1, 1, 1, 512, 512), chunktype=numpy.ndarray>

Mosaic stitch is not supported on tif files; so I will use my function relying on the PositionXYZ metadata.

## dask_image

In [6]:
from dask_image.imread import imread
imread(img_void_tile)

dask.array<_map_read_frame, shape=(12, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>

In [7]:
imread(lif)

/home/dan/workspace/imgread/.hatch/imgread/lib/python3.10/site-packages/pims/bioformats.py:50: UserWarning: loci_tools.jar not found, downloading
  warn('loci_tools.jar not found, downloading')
/home/dan/workspace/imgread/.hatch/imgread/lib/python3.10/site-packages/pims/api.py:204: UserWarning: <class 'pims.bioformats.BioformatsReader'> errored: HTTP Error 404: Not Found
  warn(message)


UnknownFormatError: All handlers returned exceptions:
<class 'pims.bioformats.BioformatsReader'> errored: HTTP Error 404: Not Found


Somehow it uses bioformats and can handle lif. No mosaic, no metadata though.

**Pycroscopy** https://pypi.org/project/pycroscopy/ is not reading lif nor ome-tif at the moment.

**large-image[all]** failed to intall.

**pyimagej** need conda?

## bioio-bioformats

In [8]:
import bioio_bioformats
%timeit bioio_bioformats.Reader(img_void_tile)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


147 ms ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
i = bioio_bioformats.Reader(img_void_tile)
i.data.shape

(4, 3, 1, 512, 512)

In [10]:
i.get_xarray_dask_stack()

<xarray.DataArray 'stack-76347f2a94c754afba35cae63eebc712' (I: 14, T: 4, C: 3,
                                                            Z: 1, Y: 512, X: 512)>
dask.array<stack, shape=(14, 4, 3, 1, 512, 512), dtype=uint16, chunksize=(1, 1, 1, 1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * C        (C) <U11 'Channel:0:0' 'Channel:0:1' 'Channel:0:2'
  * T        (T) float64 0.7301 1.985 3.241 4.496
  * Z        (Z) float64 0.0
  * Y        (Y) float64 0.0 0.2 0.4 0.6 0.8 ... 101.4 101.6 101.8 102.0 102.2
  * X        (X) float64 0.0 0.2 0.4 0.6 0.8 ... 101.4 101.6 101.8 102.0 102.2
Dimensions without coordinates: I

In [40]:
stk = i.get_dask_stack()

In [48]:
stk.A

dask.array<stack, shape=(14, 4, 3, 1, 512, 512), dtype=uint16, chunksize=(1, 1, 1, 1, 512, 512), chunktype=numpy.ndarray>

## bfio

In [49]:
import bfio
%timeit bfio.BioReader(img_void_tile)

341 ms ± 8.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
rdr = bfio.BioReader(img_void_tile)
%timeit i = rdr.read()
i = rdr.read()
i.shape

37.4 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


(512, 512, 1, 3, 4)

In [52]:
rdr.metadata

OME(
   experiments=[{'description': '', 'experimenter_ref': {'id': 'Experimenter:0ec49b9d-d35f-46d9-9adc-9526e28d4f4d'}, 'type': [<Experiment_value.FOUR_DPLUS: 'FourDPlus'>], 'id': 'Experiment:344b0000-4d68-4181-9d41-1cf0eb64969b'}],
   plates=[<1 field_type>],
   experimenters=[{'id': 'Experimenter:0ec49b9d-d35f-46d9-9adc-9526e28d4f4d'}],
   instruments=[<1 field_type>],
   images=[<14 field_type>],
   structured_annotations=[<1 field_type>],
   creator='FEI Munich GmbH, Live Acquisition, V2.5.0.6',
)

In [20]:
rdr.ps_x

(0.2, <UnitsLength.MICROMETER: 'µm'>)

## PIMS

In [57]:
import pims
%timeit fs = pims.Bioformats(img_void_tile)
fs = pims.Bioformats(img_void_tile)
fs.sizes

207 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


{'x': 512, 'y': 512, 'c': 3, 't': 4}

## PyOMETiff

In [58]:
import pyometiff
%timeit rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)
rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)

6.03 µs ± 64.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [59]:
%timeit r = rdr.read()
res = rdr.read()

38.5 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
res[2]

str

In [28]:
pyometiff.OMETIFFReader._get_metadata_template()

{'Directory': None,
 'Filename': None,
 'Extension': None,
 'ImageType': None,
 'AcqDate': None,
 'TotalSeries': None,
 'SizeX': None,
 'SizeY': None,
 'SizeZ': 1,
 'SizeC': 1,
 'SizeT': 1,
 'SizeS': 1,
 'SizeB': 1,
 'SizeM': 1,
 'PhysicalSizeX': None,
 'PhysicalSizeXUnit': None,
 'PhysicalSizeY': None,
 'PhysicalSizeYUnit': None,
 'PhysicalSizeZ': None,
 'PhysicalSizeZUnit': None,
 'Sizes BF': None,
 'DimOrder BF': None,
 'DimOrder BF Array': None,
 'ObjNA': [],
 'ObjMag': [],
 'ObjID': [],
 'ObjName': [],
 'ObjImmersion': [],
 'TubelensMag': [],
 'ObjNominalMag': [],
 'DetectorModel': [],
 'DetectorName': [],
 'DetectorID': [],
 'DetectorType': [],
 'InstrumentID': [],
 'MicroscopeType': [],
 'Channels': [],
 'ImageIDs': []}

## Final Note

I will keep 

0. Read
1. stitch
2. md_grouping


- pycromanager
- microscope
- python-microscopy

In [63]:
import imgread

In [64]:
from imgread import read

In [67]:
read.read_pims(img_void_tile)

/home/dan/workspace/imgread/.hatch/imgread/lib/python3.10/site-packages/pims/bioformats.py:410: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


({'SizeS': 14,
  'Format': 'OME-TIFF',
  'series': [{'PositionXYZ': {(50.62738, 81.83527, 21.953)}},
   {'PositionXYZ': {(50.62738, 81.73286, 21.953)}},
   {'PositionXYZ': {(50.62738, 81.63046, 21.953)}},
   {'PositionXYZ': {(50.83218, 81.52807, 21.953)}},
   {'PositionXYZ': {(50.83218, 81.42567, 21.953)}},
   {'PositionXYZ': {(50.72978, 81.52807, 21.953)}},
   {'PositionXYZ': {(50.72978, 81.42567, 21.953)}},
   {'PositionXYZ': {(50.62738, 81.52807, 21.953)}},
   {'PositionXYZ': {(50.93458, 81.52807, 21.953)}},
   {'PositionXYZ': {(50.83218, 81.63046, 21.953)}},
   {'PositionXYZ': {(50.72978, 81.63046, 21.953)}},
   {'PositionXYZ': {(50.72978, 81.73286, 21.953)}},
   {'PositionXYZ': {(50.52498, 81.73286, 21.953)}},
   {'PositionXYZ': {(50.52498, 81.63046, 21.953)}}],
  'Date': '2016-04-06T16:44:44',
  'SizeX': 512,
  'SizeY': 512,
  'SizeC': 3,
  'SizeT': 4,
  'SizeZ': 1,
  'PhysicalSizeX': 0.2,
  'PhysicalSizeY': 0.2,
  'PhysicalSizeZ': 1000.0,
  'pixel_type': '<u2'},
 <FramesSequence

In [80]:
read.read_jb(img_void_tile)

AttributeError: 'NoneType' object has no attribute 'find_class'